In [ ]:
import os
import sys

root_folder = os.path.abspath((os.getcwd()))
sys.path.append(root_folder)

import numpy as np
import numpy.linalg as la
import numpy.matlib as matl
import scipy.io as io
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import copy
%matplotlib widget

# DAGGER Analysis

In [ ]:
an = [[], [], []]
case_study = ['drone', 'freeflyer', 'rpod']
dagger_model = ['artMPC_dag0', 'artMPC_dag1', 'artMPC_dag0']# ['artMPC_dag5', 'artMPC_dag6', 'artMPC_dag9'] #cl_6, cl_7, cl_10
name_file = [root_folder + '/drone/optimization/saved_files/closed_loop/ws_analysis_DAGGER',
             root_folder + '/freeflyer/optimization/saved_files/closed_loop/0_dagger_planning_horizons/ws_analysis_DAGGER',
             root_folder + '/rpod/optimization/saved_files/closed_loop/0_dagger_planning_horizons/ws_analysis_DAGGER']
for i in range(len(an)):
    an[i].append(np.load(name_file[i] + '10.npz'))
    an[i].append(np.load(name_file[i] + '20.npz'))
    an[i].append(np.load(name_file[i] + '30.npz'))
    an[i].append(np.load(name_file[i] + '40.npz'))
    an[i].append(np.load(name_file[i] + '50.npz'))
    an[i].append(np.load(name_file[i] + '60.npz'))
    an[i].append(np.load(name_file[i] + '70.npz'))
    an[i].append(np.load(name_file[i] + '80.npz'))
    an[i].append(np.load(name_file[i] + '90.npz'))
    an[i].append(np.load(name_file[i] + '100.npz'))

In [ ]:
res_dict = {
    'ix_sample' : [],
    'ix' : [],
    'J_cvx' : [],
    'J_cvxMPC' : [],
    'J_myocvxMPC' : [],
    'J_artMPC' : [],
    'J_artMPC_dag' : [],
    'time_cvxMPC' : [],
    'time_myocvxMPC' : [],
    'time_artMPC' : [],
    'time_artMPC_dag' : [],
    'ctgs0_cvx' : [],
    'cvx_problem' : [],
    'i_unfeas_cvx' : [],
    'i_unfeas_cvxMPC' : [],
    'i_unfeas_myocvxMPC' : [],
    'i_unfeas_artMPC' : [],
    'i_unfeas_artMPC_dag' : []
}
res = {
    'freeflyer' : copy.deepcopy(res_dict),
    'drone' : copy.deepcopy(res_dict),
    'rpod' : copy.deepcopy(res_dict)
}

for n_case in range(len(an)):
    for i in range(len(an[n_case])):
        results = res[case_study[n_case]]
        results['ix_sample'].append(an[n_case][i]['test_dataset_ix'])
        results['ix'].append(np.arange(len(an[n_case][i]['test_dataset_ix'])))
        results['J_cvx'].append(an[n_case][i]['J_cvx'])
        results['J_cvxMPC'].append(an[n_case][i]['J_cvxMPC'])
        results['J_myocvxMPC'].append(an[n_case][i]['J_myocvxMPC'])
        results['J_artMPC'].append(an[n_case][i]['J_artMPC'])
        results['J_artMPC_dag'].append(an[n_case][i]['J_'+dagger_model[n_case]])
        results['time_cvxMPC'].append(an[n_case][i]['time_cvxMPC'])
        results['time_myocvxMPC'].append(an[n_case][i]['time_myocvxMPC'])
        results['time_artMPC'].append(an[n_case][i]['time_artMPC'])
        results['time_artMPC_dag'].append(an[n_case][i]['time_'+dagger_model[n_case]])
        results['ctgs0_cvx'].append(an[n_case][i]['ctgs0_cvx'])
        results['cvx_problem'].append(an[n_case][i]['cvx_problem'])
        results['i_unfeas_cvx'].append(an[n_case][i]['i_unfeas_cvx'])
        results['i_unfeas_cvxMPC'].append(an[n_case][i]['i_unfeas_cvxMPC'])
        results['i_unfeas_myocvxMPC'].append(an[n_case][i]['i_unfeas_myocvxMPC'])
        results['i_unfeas_artMPC'].append(an[n_case][i]['i_unfeas_artMPC'])
        results['i_unfeas_artMPC_dag'].append(an[n_case][i]['i_unfeas_'+dagger_model[n_case]])

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        print('Unfeasible cvx: ', len(results['i_unfeas_cvx'][i]))
        print('Unfeasible cvxMPC: ', len(results['i_unfeas_cvxMPC'][i]))
        print('Unfeasible myocvxMPC: ', len(results['i_unfeas_myocvxMPC'][i]))
        print('Unfeasible artMPC: ', len(results['i_unfeas_artMPC'][i]))
        print('Unfeasible artMPC_dag: ', len(results['i_unfeas_artMPC_dag'][i]))

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    results['ix_ctgs0_ge_1_comparison'] = []
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        ix_unfeas = set(results['i_unfeas_cvx'][i]).union(set(results['i_unfeas_cvxMPC'][i])).union(set(results['i_unfeas_artMPC'][i])).union(set(results['i_unfeas_artMPC_dag'][i])).union(set(results['i_unfeas_myocvxMPC'][i]))
        ix_feas_comparison = set(results['ix'][i]) - ix_unfeas - set([-1])
        print('Feasible samples for comparison', len(ix_feas_comparison))
        ctgs_selector = (results['ctgs0_cvx'][i] >= 1) & (results['ctgs0_cvx'][i] <= 50)
        ix_ctgs0_ge_1 = set(results['ix'][i][ctgs_selector])
        print('Samples with ctgs0 of interest', len(ix_ctgs0_ge_1))
        results['ix_ctgs0_ge_1_comparison'].append(ix_feas_comparison.intersection(ix_ctgs0_ge_1))
        print('Samples with ctgs0 of insterest for comparison', len(results['ix_ctgs0_ge_1_comparison'][i]))

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]    
    results['mean_costs_list'] = []
    results['mean_time_list'] = []
    results['ctgs0_set'] = []
    results['ctgs0_count'] = []
    scale = 1000
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        # Selection for comparison
        J_cvx_comp = results['J_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_cvxMPC_comp = results['J_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_myocvxMPC_comp = results['J_myocvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_comp = results['J_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_dag_comp = results['J_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        time_cvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_myocvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_comp = results['time_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_dag_comp = results['time_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        ctgs0_cvx_comp = results['ctgs0_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()

        # Ordered results for ctgs0
        results['ctgs0_set'].append(set(ctgs0_cvx_comp))
        results['mean_costs_list'].append({
            'cvx' : np.zeros((len(results['ctgs0_set'][i],))),
            'cvxMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'myocvxMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'artMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'artMPC_dag' : np.zeros((len(results['ctgs0_set'][i],)))
        })
        results['mean_time_list'].append({
            'cvxMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'myocvxMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'artMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'artMPC_dag' : np.zeros((len(results['ctgs0_set'][i]),100))
        })
        results['ctgs0_count'].append(np.zeros((len(results['ctgs0_set'][i],))))

        for j,ctg in enumerate(results['ctgs0_set'][i]):
            idx_ctg = ctgs0_cvx_comp >= ctg
            #idx_ctg = (ctgs0_cvx_comp >= np.floor(ctg/10)*10) & (ctgs0_cvx_comp <= (np.floor(ctg/10) + 1)*10)
            results['mean_costs_list'][i]['cvx'][j] = J_cvx_comp[idx_ctg].mean()

            results['mean_costs_list'][i]['cvxMPC'][j] = J_cvxMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['myocvxMPC'][j] = J_myocvxMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['artMPC'][j] = J_artMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['artMPC_dag'][j] = J_artMPC_dag_comp[idx_ctg].mean()

            # Time
            results['mean_time_list'][i]['cvxMPC'][j] = time_cvxMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['myocvxMPC'][j] = time_myocvxMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['artMPC'][j] = time_artMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['artMPC_dag'][j] = time_artMPC_dag_comp[idx_ctg].mean(axis=0)
            results['ctgs0_count'][i][j] = idx_ctg.sum()

# Models comparison
1) Plot comparing the performances of different models as a function of the ctg0 of the maneuver, keeping the planning horizon H fixed

In [ ]:
ctgs0_list = []
n_case = 0
plt.style.use('default')#('seaborn-v0_8-colorblind')
for i, mean_costs in enumerate(res[case_study[n_case]]['mean_costs_list']):
    print('==================================\nDAGGER --- Closed-loop analysis', i)
    ctgs0_list.append(list(res[case_study[n_case]]['ctgs0_set'][i]))
    #mpl.style.use('seaborn-v0_8-colorblind')
    fig, ax1 = plt.subplots(figsize=(8,5))
    box = ax1.get_position()
    ax1.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
    ax1.set_title('Planning horizon ' + str(i*10 + 10))
    '''ax2 = ax1.twinx()
    ax2.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
    ax2.plot(ctgs0_list[i], ctgs0_count[i],c='C0')
    ax2.tick_params(axis='y', labelcolor='C0')
    ax2.set_ylabel('#Samples', color='C0')'''
    ax1.plot(ctgs0_list[i], (mean_costs['cvxMPC'] - mean_costs['cvx'])*scale, marker='o', label='cvxMPC')
    ax1.plot(ctgs0_list[i], (mean_costs['myocvxMPC'] - mean_costs['cvx'])*scale, marker='d', label='myocvxMPC')
    ax1.plot(ctgs0_list[i], (mean_costs['artMPC'] - mean_costs['cvx'])*scale, marker='s', label='artMPC')
    ax1.plot(ctgs0_list[i], (mean_costs['artMPC_dag'] - mean_costs['cvx'])*scale, color='C3', marker='^', label='artMPC_dag')
    ax1.plot(ctgs0_list[i], (mean_costs['cvx'] - mean_costs['cvx'])*scale,'k--',label='Lower bound')
    ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=4)
    ax1.set_xlabel('$ctgs_0$')
    ax1.set_ylabel('$J - J_{cvx}$ [mm/s]', c='#D93637')
    ax1.tick_params(axis='y', labelcolor='#D93637')

plt.show()

3) Box plot per la distribuzione su tutto il dataset e non mediando l'andamento mediato per ctgs

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    results['h_dataset'] = [[],[],[],[]]
    results['time_dataset'] = [[],[],[],[]]
    results['mean_trend'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['std_trend'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['first_quart'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['third_quart'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        # Selection for comparison
        J_cvx_comp = results['J_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_cvxMPC_comp = results['J_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_myocvxMPC_comp = results['J_myocvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_comp = results['J_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_dag_comp = results['J_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        time_cvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_myocvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_comp = results['time_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_dag_comp = results['time_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        ctgs0_cvx_comp = results['ctgs0_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()

        # convexMPC
        results['h_dataset'][0].append((J_cvxMPC_comp - J_cvx_comp))
        results['mean_trend'][0,i] = ((J_cvxMPC_comp - J_cvx_comp)).mean()
        results['first_quart'][0,i] = np.percentile((J_cvxMPC_comp - J_cvx_comp), 25)
        results['third_quart'][0,i] = np.percentile((J_cvxMPC_comp - J_cvx_comp), 75)
        results['time_dataset'][0].append(time_cvxMPC_comp[:,:(-(i+1)*10)+1].max(axis=1))#.max(axis=1))
        results['std_trend'][0,i] = ((J_cvxMPC_comp - J_cvx_comp)).std()
        # myopic cvxMPC
        results['h_dataset'][1].append((J_myocvxMPC_comp - J_cvx_comp))
        results['mean_trend'][1,i] = ((J_myocvxMPC_comp - J_cvx_comp)).mean()
        results['first_quart'][1,i] = np.percentile((J_myocvxMPC_comp - J_cvx_comp), 25)
        results['third_quart'][1,i] = np.percentile((J_myocvxMPC_comp - J_cvx_comp), 75)
        results['time_dataset'][1].append(time_myocvxMPC_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['std_trend'][1,i] = ((J_myocvxMPC_comp - J_cvx_comp)).std()
        # artMPC
        results['h_dataset'][2].append((J_artMPC_comp - J_cvx_comp))
        results['mean_trend'][2,i] = ((J_artMPC_comp - J_cvx_comp)).mean()
        results['first_quart'][2,i] = np.percentile((J_artMPC_comp - J_cvx_comp), 25)
        results['third_quart'][2,i] = np.percentile((J_artMPC_comp - J_cvx_comp), 75)
        results['time_dataset'][2].append(time_artMPC_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['std_trend'][2,i] = ((J_artMPC_comp - J_cvx_comp)).std()
        # artMPC_dag
        results['h_dataset'][3].append((J_artMPC_dag_comp - J_cvx_comp))
        results['mean_trend'][3,i] = ((J_artMPC_dag_comp - J_cvx_comp)).mean()
        results['first_quart'][3,i] = np.percentile((J_artMPC_dag_comp - J_cvx_comp), 25)
        results['third_quart'][3,i] = np.percentile((J_artMPC_dag_comp - J_cvx_comp), 75)
        results['time_dataset'][3].append(time_artMPC_dag_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['std_trend'][3,i] = ((J_artMPC_dag_comp - J_cvx_comp)).std()

In [ ]:
fig, ax1 = plt.subplots(1,3,figsize=(20,3))
plt.style.use('seaborn-v0_8-colorblind')
for n_case in range(len(an)):
    results = res[case_study[n_case]]
    planning_horizons = np.arange(10, len(results['mean_costs_list'])*10+1, 10)
    i_plot_0 = 0
    scale = 1

    lowerbound = np.zeros((4,len(planning_horizons)))
    upperbound = np.zeros((4,len(planning_horizons)))
    for i in range(lowerbound.shape[0]):
        lowerbound[i] = results['mean_trend'][i] - results['std_trend'][i]
        upperbound[i] = results['mean_trend'][i] + results['std_trend'][i]
    ax1[n_case].plot(planning_horizons[i_plot_0:], (results['mean_trend'][0,i_plot_0:]/results['mean_trend'][1,0])*scale, c='C0', marker='o', label='cvxMPC')
    #ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[0,i_plot_0:])*scale, (upperbound[0,i_plot_0:])*scale, alpha=0.2, facecolor='C0')
    ax1[n_case].plot(planning_horizons[i_plot_0:], (results['mean_trend'][1,i_plot_0:]/results['mean_trend'][1,0])*scale, c='k', marker='d', label='myocvxMPC')
    #ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[1,i_plot_0:])*scale, (upperbound[1,i_plot_0:])*scale, alpha=0.2, facecolor='C1')
    ax1[n_case].plot(planning_horizons[i_plot_0:], (results['mean_trend'][2,i_plot_0:]/results['mean_trend'][1,0])*scale, c='indianred', marker='s', label='artMPC')
    #ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[2,i_plot_0:])*scale, (upperbound[2,i_plot_0:])*scale, alpha=0.2, facecolor='indianred')
    ax1[n_case].plot(planning_horizons[i_plot_0:], (results['mean_trend'][3,i_plot_0:]/results['mean_trend'][1,0])*scale, c='C1', marker = '^', label='artMPC_dag')
    #ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[3,i_plot_0:])*scale, (upperbound[3,i_plot_0:])*scale, alpha=0.2, facecolor='C1')
    ax1[n_case].plot(planning_horizons[i_plot_0:], np.zeros_like(planning_horizons[i_plot_0:])*scale,'k--',label='Lower bound')
    if n_case == 0:
        ax1[n_case].legend(loc='upper right')#, bbox_to_anchor=(0.5, -0.1), ncol=3)
    if n_case+1 == len(case_study):
        ax1[n_case].set_xlabel('$H_{plan}$')
        ax1[n_case].set_xticks([10,20,30,40,50,60,70,80,90,100])
    else:
        ax1[n_case].set_xticklabels([])
    ax1[n_case].set_ylabel(r'$\frac{J - J_{cvx}}{\Delta J_{norm}}$   [-]', fontsize=13)
    ax1[n_case].tick_params(axis='y')
    '''ax1.set_yscale('log')'''
fig.savefig('closed_loop_cost_per_applications.svg')

In [ ]:
# Mean trend cross application
model_norm = 1
mean_trend_aggr = {
    'cvxMPC' : (res[case_study[0]]['mean_trend'][0]/max(res[case_study[0]]['mean_trend'][model_norm,:]) + res[case_study[1]]['mean_trend'][0]/max(res[case_study[1]]['mean_trend'][model_norm,:]) + res[case_study[2]]['mean_trend'][0]/max(res[case_study[2]]['mean_trend'][model_norm,:]))/3,
    'myocvxMPC' : (res[case_study[0]]['mean_trend'][1]/max(res[case_study[0]]['mean_trend'][model_norm,:]) + res[case_study[1]]['mean_trend'][1]/max(res[case_study[1]]['mean_trend'][model_norm,:]) + res[case_study[2]]['mean_trend'][1]/max(res[case_study[2]]['mean_trend'][model_norm,:]))/3,
    'artMPC' : (res[case_study[0]]['mean_trend'][2]/max(res[case_study[0]]['mean_trend'][model_norm,:]) + res[case_study[1]]['mean_trend'][2]/max(res[case_study[1]]['mean_trend'][model_norm,:]) + res[case_study[2]]['mean_trend'][2]/max(res[case_study[2]]['mean_trend'][model_norm,:]))/3,
    'artMPC_dag' : (res[case_study[0]]['mean_trend'][3]/max(res[case_study[0]]['mean_trend'][model_norm,:]) + res[case_study[1]]['mean_trend'][3]/max(res[case_study[1]]['mean_trend'][model_norm,:]) + res[case_study[2]]['mean_trend'][3]/max(res[case_study[2]]['mean_trend'][model_norm,:]))/3
}

fig, ax1 = plt.subplots(figsize=(10,5))
box = ax1.get_position()
ax1.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['cvxMPC'][i_plot_0:], c='C0', marker='o', label='cvxMPC')
#ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[0,i_plot_0:])*scale, (upperbound[0,i_plot_0:])*scale, alpha=0.2, facecolor='C0')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['myocvxMPC'][i_plot_0:], c='C1', marker='d', label='myocvxMPC')
#ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[1,i_plot_0:])*scale, (upperbound[1,i_plot_0:])*scale, alpha=0.2, facecolor='C1')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC'][i_plot_0:], c='C2', marker='s', label='artMPC')
#ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[2,i_plot_0:])*scale, (upperbound[2,i_plot_0:])*scale, alpha=0.2, facecolor='C2')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC_dag'][i_plot_0:], c='C3', marker = '^', label='artMPC_dag')
#ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[3,i_plot_0:])*scale, (upperbound[3,i_plot_0:])*scale, alpha=0.2, facecolor='C4')
ax1.plot(planning_horizons[i_plot_0:], np.zeros_like(planning_horizons[i_plot_0:])*scale,'k--',label='Lower bound')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, 1.15), ncol=5)
ax1.set_xlabel('$H_{plan}$')
ax1.set_ylabel(r'$\bar{\left(J_{RHC} - J_{OL-cvx}\right)}$ [-]', fontsize=13)
ax1.tick_params(axis='y')
#ax1.set_yscale('log')
#fig.savefig('closed_loop_cost_cross_applications.svg')

In [ ]:
aggr_curves = {
    'cvxMPC' : np.vstack((res[case_study[0]]['mean_trend'][0]/max(res[case_study[0]]['mean_trend'][model_norm]), res[case_study[1]]['mean_trend'][0]/max(res[case_study[1]]['mean_trend'][model_norm]), res[case_study[2]]['mean_trend'][0]/max(res[case_study[2]]['mean_trend'][model_norm]))),
    'myocvxMPC' : np.vstack((res[case_study[0]]['mean_trend'][1]/max(res[case_study[0]]['mean_trend'][model_norm]), res[case_study[1]]['mean_trend'][1]/max(res[case_study[1]]['mean_trend'][model_norm]), res[case_study[2]]['mean_trend'][1]/max(res[case_study[2]]['mean_trend'][model_norm]))),
    'artMPC' : np.vstack((res[case_study[0]]['mean_trend'][2]/max(res[case_study[0]]['mean_trend'][model_norm]), res[case_study[1]]['mean_trend'][2]/max(res[case_study[1]]['mean_trend'][model_norm]), res[case_study[2]]['mean_trend'][2]/max(res[case_study[2]]['mean_trend'][model_norm]))),
    'artMPC_dag' : np.vstack((res[case_study[0]]['mean_trend'][3]/max(res[case_study[0]]['mean_trend'][model_norm]), res[case_study[1]]['mean_trend'][3]/max(res[case_study[1]]['mean_trend'][model_norm]), res[case_study[2]]['mean_trend'][3]/max(res[case_study[2]]['mean_trend'][model_norm])))
}
mean_trend_aggr = {
    'cvxMPC' : aggr_curves['cvxMPC'].mean(axis=0),
    'myocvxMPC' : aggr_curves['myocvxMPC'].mean(axis=0),
    'artMPC' : aggr_curves['artMPC'].mean(axis=0),
    'artMPC_dag' : aggr_curves['artMPC_dag'].mean(axis=0)
}
std_trend_aggr = {
    'cvxMPC' : aggr_curves['cvxMPC'].std(axis=0),
    'myocvxMPC' : aggr_curves['myocvxMPC'].std(axis=0),
    'artMPC' : aggr_curves['artMPC'].std(axis=0),
    'artMPC_dag' : aggr_curves['artMPC_dag'].std(axis=0)
}

lowerbound = np.zeros((4,len(planning_horizons)))
upperbound = np.zeros((4,len(planning_horizons)))
for i in range(lowerbound.shape[0]):
    key = list(mean_trend_aggr.keys())[i]
    lowerbound[i] = mean_trend_aggr[key] - 0.5*std_trend_aggr[key]
    upperbound[i] = mean_trend_aggr[key] + 0.5*std_trend_aggr[key]

sns_red = (0.7686274509803922, 0.3058823529411765, 0.3215686274509804)
l_width = 2.5
plt.style.use('seaborn-v0_8-colorblind')
fig, ax1 = plt.subplots(figsize=(8,4))
box = ax1.get_position()
ax1.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['cvxMPC'][i_plot_0:], c='C0', linewidth=l_width, marker='o', label='REL-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[0,i_plot_0:])*scale, (upperbound[0,i_plot_0:])*scale, alpha=0.1, facecolor='C0')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['myocvxMPC'][i_plot_0:], c='k', linewidth=l_width, marker='d', label='DIST-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[1,i_plot_0:])*scale, (upperbound[1,i_plot_0:])*scale, alpha=0.1, facecolor='k')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC'][i_plot_0:], c=sns_red, linewidth=l_width, marker='s', label='TTO-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[2,i_plot_0:])*scale, (upperbound[2,i_plot_0:])*scale, alpha=0.15, facecolor=sns_red)
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC_dag'][i_plot_0:], c='C1', linewidth=l_width, marker = '^', label='FT-TTO-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[3,i_plot_0:])*scale, (upperbound[3,i_plot_0:])*scale, alpha=0.2, facecolor='C1')
ax1.plot(planning_horizons[i_plot_0:], np.zeros_like(planning_horizons[i_plot_0:])*scale,'k--',label='Lower bound')
ax1.legend(loc='upper right', fontsize=12)#, bbox_to_anchor=(0.5, 1.15))
ax1.set_xlabel('Planning horizon', fontsize=13)
ax1.set_ylabel(r'$\frac{J - J_{LB}}{\Delta J_{MAX}}$   [-]', fontsize=15)
ax1.tick_params(axis='y')
ax1.set_xticks(np.arange(10,101,10))
#ax1.set_yscale('log')
fig.savefig('closed_loop_cost_cross_applications.svg')

# Time comparison


In [ ]:
an = [[], [], []]
case_study = ['car', 'drone', 'rpod']
dagger_model = ['artMPC_dag5', 'artMPC_dag6', 'artMPC_dag9'] #cl_6, cl_7, cl_10
name_file = [root_folder + '/car/optimization/saved_files/closed_loop/2_dagger_time_analysis_final/ws_analysis_time_DAGGER',
             root_folder + '/drone/optimization/saved_files/closed_loop/3_dagger_time_analysis final/ws_analysis_time_DAGGER',
             root_folder + '/rpod/optimization/saved_files/closed_loop/0_dagger_time_planning_horizons/ws_analysis_time_DAGGER']#0_dagger_time_planning_horizons/ws_analysis_time_DAGGER']
for i in range(len(an)):
    an[i].append(np.load(name_file[i] + '10.npz'))
    an[i].append(np.load(name_file[i] + '20.npz'))
    an[i].append(np.load(name_file[i] + '30.npz'))
    an[i].append(np.load(name_file[i] + '40.npz'))
    an[i].append(np.load(name_file[i] + '50.npz'))
    an[i].append(np.load(name_file[i] + '60.npz'))
    an[i].append(np.load(name_file[i] + '70.npz'))
    an[i].append(np.load(name_file[i] + '80.npz'))
    an[i].append(np.load(name_file[i] + '90.npz'))
    an[i].append(np.load(name_file[i] + '100.npz'))

In [ ]:
res_dict = {
    'ix_sample' : [],
    'ix' : [],
    'J_cvx' : [],
    'J_cvxMPC' : [],
    'J_myocvxMPC' : [],
    'J_artMPC' : [],
    'J_artMPC_dag' : [],
    'time_cvxMPC' : [],
    'time_myocvxMPC' : [],
    'time_artMPC' : [],
    'time_artMPC_dag' : [],
    'ctgs0_cvx' : [],
    'cvx_problem' : [],
    'i_unfeas_cvx' : [],
    'i_unfeas_cvxMPC' : [],
    'i_unfeas_myocvxMPC' : [],
    'i_unfeas_artMPC' : [],
    'i_unfeas_artMPC_dag' : []
}
res = {
    'car' : copy.deepcopy(res_dict),
    'drone' : copy.deepcopy(res_dict),
    'rpod' : copy.deepcopy(res_dict)
}

for n_case in range(len(an)):
    for i in range(len(an[n_case])):
        results = res[case_study[n_case]]
        results['ix_sample'].append(an[n_case][i]['test_dataset_ix'])
        results['ix'].append(np.arange(len(an[n_case][i]['test_dataset_ix'])))
        results['J_cvx'].append(an[n_case][i]['J_cvx'])
        results['J_cvxMPC'].append(an[n_case][i]['J_cvxMPC'])
        results['J_myocvxMPC'].append(an[n_case][i]['J_myocvxMPC'])
        results['J_artMPC'].append(an[n_case][i]['J_artMPC'])
        results['J_artMPC_dag'].append(an[n_case][i]['J_'+dagger_model[n_case]])
        results['time_cvxMPC'].append(an[n_case][i]['time_cvxMPC'])
        results['time_myocvxMPC'].append(an[n_case][i]['time_myocvxMPC'])
        results['time_artMPC'].append(an[n_case][i]['time_artMPC'])
        results['time_artMPC_dag'].append(an[n_case][i]['time_'+dagger_model[n_case]])
        results['ctgs0_cvx'].append(an[n_case][i]['ctgs0_cvx'])
        results['cvx_problem'].append(an[n_case][i]['cvx_problem'])
        results['i_unfeas_cvx'].append(an[n_case][i]['i_unfeas_cvx'])
        results['i_unfeas_cvxMPC'].append(an[n_case][i]['i_unfeas_cvxMPC'])
        results['i_unfeas_myocvxMPC'].append(an[n_case][i]['i_unfeas_myocvxMPC'])
        results['i_unfeas_artMPC'].append(an[n_case][i]['i_unfeas_artMPC'])
        results['i_unfeas_artMPC_dag'].append(an[n_case][i]['i_unfeas_'+dagger_model[n_case]])

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        print('Unfeasible cvx: ', len(results['i_unfeas_cvx'][i]))
        print('Unfeasible cvxMPC: ', len(results['i_unfeas_cvxMPC'][i]))
        print('Unfeasible myocvxMPC: ', len(results['i_unfeas_myocvxMPC'][i]))
        print('Unfeasible artMPC: ', len(results['i_unfeas_artMPC'][i]))
        print('Unfeasible artMPC_dag: ', len(results['i_unfeas_artMPC_dag'][i]))

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    results['ix_ctgs0_ge_1_comparison'] = []
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        ix_unfeas = set(results['i_unfeas_cvx'][i]).union(set(results['i_unfeas_cvxMPC'][i])).union(set(results['i_unfeas_artMPC'][i])).union(set(results['i_unfeas_artMPC_dag'][i])).union(set(results['i_unfeas_myocvxMPC'][i]))
        ix_feas_comparison = set(results['ix'][i]) - ix_unfeas - set([-1])
        print('Feasible samples for comparison', len(ix_feas_comparison))
        ctgs_selector = (results['ctgs0_cvx'][i] >= 1) & (results['ctgs0_cvx'][i] <= 50)
        ix_ctgs0_ge_1 = set(results['ix'][i][ctgs_selector])
        print('Samples with ctgs0 of interest', len(ix_ctgs0_ge_1))
        results['ix_ctgs0_ge_1_comparison'].append(ix_feas_comparison.intersection(ix_ctgs0_ge_1))
        print('Samples with ctgs0 of insterest for comparison', len(results['ix_ctgs0_ge_1_comparison'][i]))

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]    
    results['mean_costs_list'] = []
    results['mean_time_list'] = []
    results['ctgs0_set'] = []
    results['ctgs0_count'] = []
    scale = 1000
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        # Selection for comparison
        J_cvx_comp = results['J_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_cvxMPC_comp = results['J_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_myocvxMPC_comp = results['J_myocvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_comp = results['J_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_dag_comp = results['J_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        time_cvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_myocvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_comp = results['time_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_dag_comp = results['time_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        ctgs0_cvx_comp = results['ctgs0_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()

        # Ordered results for ctgs0
        results['ctgs0_set'].append(set(ctgs0_cvx_comp))
        results['mean_costs_list'].append({
            'cvx' : np.zeros((len(results['ctgs0_set'][i],))),
            'cvxMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'myocvxMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'artMPC' : np.zeros((len(results['ctgs0_set'][i],))),
            'artMPC_dag' : np.zeros((len(results['ctgs0_set'][i],)))
        })
        results['mean_time_list'].append({
            'cvxMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'myocvxMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'artMPC' : np.zeros((len(results['ctgs0_set'][i]),100)),
            'artMPC_dag' : np.zeros((len(results['ctgs0_set'][i]),100))
        })
        results['ctgs0_count'].append(np.zeros((len(results['ctgs0_set'][i],))))

        for j,ctg in enumerate(results['ctgs0_set'][i]):
            idx_ctg = ctgs0_cvx_comp >= ctg
            #idx_ctg = (ctgs0_cvx_comp >= np.floor(ctg/10)*10) & (ctgs0_cvx_comp <= (np.floor(ctg/10) + 1)*10)
            results['mean_costs_list'][i]['cvx'][j] = J_cvx_comp[idx_ctg].mean()

            results['mean_costs_list'][i]['cvxMPC'][j] = J_cvxMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['myocvxMPC'][j] = J_myocvxMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['artMPC'][j] = J_artMPC_comp[idx_ctg].mean()
            results['mean_costs_list'][i]['artMPC_dag'][j] = J_artMPC_dag_comp[idx_ctg].mean()

            # Time
            results['mean_time_list'][i]['cvxMPC'][j] = time_cvxMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['myocvxMPC'][j] = time_myocvxMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['artMPC'][j] = time_artMPC_comp[idx_ctg].mean(axis=0)
            results['mean_time_list'][i]['artMPC_dag'][j] = time_artMPC_dag_comp[idx_ctg].mean(axis=0)
            results['ctgs0_count'][i][j] = idx_ctg.sum()

# Models comparison
1) Plot comparing the performances of different models as a function of the ctg0 of the maneuver, keeping the planning horizon H fixed

3) Box plot per la distribuzione su tutto il dataset e non mediando l'andamento mediato per ctgs

In [ ]:
for n_case in range(len(an)):
    print('\n=====================\nCase study : '+ case_study[n_case])
    results = res[case_study[n_case]]
    results['h_dataset'] = [[],[],[],[]]
    results['time_dataset'] = [[],[],[],[]]
    results['mean_trend'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['std_trend'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['first_quart'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    results['third_quart'] = np.zeros((len(results['h_dataset']),len(results['mean_costs_list'])))
    for i in range(len(an[n_case])):
        print('------------------\nDAGGER --- planning horizon = ', 10*(i+1))
        # Selection for comparison
        J_cvx_comp = results['J_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_cvxMPC_comp = results['J_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_myocvxMPC_comp = results['J_myocvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_comp = results['J_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        J_artMPC_dag_comp = results['J_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()
        time_cvxMPC_comp = results['time_cvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_myocvxMPC_comp = results['time_myocvxMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_comp = results['time_artMPC'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        time_artMPC_dag_comp = results['time_artMPC_dag'][i][list(results['ix_ctgs0_ge_1_comparison'][i]),:].copy()
        ctgs0_cvx_comp = results['ctgs0_cvx'][i][list(results['ix_ctgs0_ge_1_comparison'][i])].copy()

        # convexMPC
        #results['time_dataset'][0].append(time_cvxMPC_comp[:,:(-(i+1)*10)+1].max(axis=1))#.max(axis=1))
        results['time_dataset'][0].append(time_cvxMPC_comp.max(axis=1))#.max(axis=1))
        results['mean_trend'][0,i] = (time_cvxMPC_comp.max(axis=1)).mean()
        results['std_trend'][0,i] = (time_cvxMPC_comp.max(axis=1)).std()
        results['first_quart'][0,i] = np.percentile(time_cvxMPC_comp.max(axis=1), 25)
        results['third_quart'][0,i] = np.percentile(time_cvxMPC_comp.max(axis=1), 75)
        # myopic cvxMPC
        #results['time_dataset'][1].append(time_myocvxMPC_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['time_dataset'][1].append(time_myocvxMPC_comp.max(axis=1))#.max(axis=1))
        results['mean_trend'][1,i] = (time_myocvxMPC_comp.max(axis=1)).mean()
        results['std_trend'][1,i] = (time_myocvxMPC_comp.max(axis=1)).std()
        results['first_quart'][1,i] = np.percentile(time_myocvxMPC_comp.max(axis=1), 25)
        results['third_quart'][1,i] = np.percentile(time_myocvxMPC_comp.max(axis=1), 75)
        # artMPC
        #results['time_dataset'][2].append(time_artMPC_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['time_dataset'][2].append(time_artMPC_comp.max(axis=1))#.max(axis=1))
        results['mean_trend'][2,i] = (time_artMPC_comp.max(axis=1)).mean()
        results['std_trend'][2,i] = (time_artMPC_comp.max(axis=1)).std()
        results['first_quart'][2,i] = np.percentile(time_artMPC_comp.max(axis=1), 25)
        results['third_quart'][2,i] = np.percentile(time_artMPC_comp.max(axis=1), 75)
        # artMPC_dag
        #results['time_dataset'][3].append(time_artMPC_dag_comp[:,:(-(i+1)*10)+1].mean(axis=1))#.max(axis=1))
        results['time_dataset'][3].append(time_artMPC_dag_comp.max(axis=1))#.max(axis=1))
        results['mean_trend'][3,i] = (time_artMPC_dag_comp.max(axis=1)).mean()
        results['std_trend'][3,i] = (time_artMPC_dag_comp.max(axis=1)).std()
        results['first_quart'][3,i] = np.percentile(time_artMPC_dag_comp.max(axis=1), 25)
        results['third_quart'][3,i] = np.percentile(time_artMPC_dag_comp.max(axis=1), 75)

In [ ]:
len(res[case_study[n_case]]['time_dataset'])

In [ ]:
fig, ax = plt.subplots()
n_case = 0
ax.plot(res[case_study[n_case]]['time_cvxMPC'][0].T)
plt.show()

In [ ]:
H_labels = [str(i) for i in np.arange(10,101,10)]
n_case = 0
results = res[case_study[n_case]]
fig, ax1 = plt.subplots(figsize=(20,10))
box = ax1.get_position()
ax1.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
b_cvxMPC = ax1.boxplot(results['time_dataset'][0], positions=np.array(np.arange(len(results['time_dataset'][0])))*3.0-0.65, widths=0.6, notch=False, vert=True, sym='', patch_artist=True, labels=H_labels, showmeans=True)
b_artMPC = ax1.boxplot(results['time_dataset'][2], positions=np.array(np.arange(len(results['time_dataset'][2])))*3.0+0., widths=0.6, notch=False, vert=True, sym='', patch_artist=True, labels=H_labels, showmeans=True)
b_artMPC_dag_Hr = ax1.boxplot(results['time_dataset'][3], positions=np.array(np.arange(len(results['time_dataset'][3])))*3.0+0.65, widths=0.6, notch=False, vert=True, sym='', patch_artist=True, labels=H_labels, showmeans=True)

# each plot returns a dictionary, use plt.setp()
# function to assign the color code
# for all properties of the box plot of particular group
# use the below function to set color for particular group,
# by iterating over all properties of the box plot
def define_box_properties(plot_name, color_code, face_color, label):
    for k, v in plot_name.items():
        plt.setp(plot_name.get(k), color=color_code)
    for patch in plot_name['boxes']:
        patch.set_facecolor(face_color)
    '''for median in plot_name['medians']:
        median.set_c('w')'''
    # use plot function to draw a small line to name the legend.
    plt.plot([], c=color_code, label=label)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), ncol=3)
 
 
# setting colors for each groups
define_box_properties(b_cvxMPC, 'orangered', 'lightsalmon', 'cvxMPC')
define_box_properties(b_artMPC, 'darkred', 'lightcoral', 'artMPC')
define_box_properties(b_artMPC_dag_Hr, 'darkorange', 'navajowhite', 'artMPC_dag_Hr')

# set the x label values
plt.xticks(np.arange(0, len(H_labels) * 3, 3), H_labels)
 
# set the limit for x axis
plt.xlim(-2, len(H_labels)*3-1)
ax1.set_xlabel('$H_{plan}$')
ax1.set_ylabel(r'$\frac{J - J_{cvx}}{J_{cvx}}$ [%]', color='#D93637', fontsize=13)
ax1.tick_params(axis='y', labelcolor='#D93637')
#ax1.set_yscale('log')

In [ ]:
plt.style.use('seaborn-v0_8-colorblind')
mean_time = [[td.mean() for td in tdm_method] for tdm_method in results['time_dataset']]
std_time = [[td.std() for td in tdm_method] for tdm_method in results['time_dataset']]
for i in range(4):
    if len(mean_time[i]) == 0:
        mean_time[i] = [np.nan]*len(H_labels)
        std_time[i] = [np.nan]*len(H_labels)
mean_time = np.array(mean_time)
std_time = np.array(std_time)

lowerbound = np.zeros((4,len(H_labels)))
upperbound = np.zeros((4,len(H_labels)))
for i in range(lowerbound.shape[0]):
    lowerbound[i] = mean_time[i] - std_time[i]
    upperbound[i] = mean_time[i] + std_time[i]
    
fig, ax = plt.subplots()
ax.plot(np.arange(10,101,10),mean_time[0],c='C0',label='cvxMPC')
ax.fill_between(np.arange(10,101,10),lowerbound[0],upperbound[0],facecolor='C0',alpha=0.1)
'''ax.plot(np.arange(10,101,10),mean_time[1],label='myocvxMPC')
ax.plot(np.arange(10,101,10),mean_time[2],label='artMPC')
ax.fill_between(np.arange(10,101,10),lowerbound[2],upperbound[2],facecolor='C2',alpha=0.2)'''
ax.plot(np.arange(10,101,10),mean_time[3],c='C1',label='artMPC_dag')
ax.fill_between(np.arange(10,101,10),lowerbound[3],upperbound[3],facecolor='C1',alpha=0.15)
#ax.plot(np.arange(10,101,10), 0.1*np.ones(10,),'k--',label='0.1 s')
ax.set_xlabel('$H_{plan}$')
ax.set_ylabel('Computational time [s]')
ax.legend(loc='upper left')
ax.set_title(case_study[n_case])
#ax.set_yscale('log')
fig.savefig('')
plt.show()

In [ ]:
fig, ax1 = plt.subplots(3,1,figsize=(10,15))
plt.style.use('seaborn-v0_8-colorblind')
for n_case in range(len(an)):
    results = res[case_study[n_case]]
    planning_horizons = np.arange(10, len(results['mean_costs_list'])*10+1, 10)
    i_plot_0 = 0
    scale = 1

    lowerbound = np.zeros((4,len(planning_horizons)))
    upperbound = np.zeros((4,len(planning_horizons)))
    for i in range(lowerbound.shape[0]):
        lowerbound[i] = results['mean_trend'][i] - results['std_trend'][i]
        upperbound[i] = results['mean_trend'][i] + results['std_trend'][i]
    
    ax1[n_case].plot(planning_horizons[i_plot_0:], results['mean_trend'][0,i_plot_0:], c='C0', marker='o', label='cvxMPC')
    ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[0,i_plot_0:])*scale, (upperbound[0,i_plot_0:])*scale, alpha=0.2, facecolor='C0')
    ax1[n_case].plot(planning_horizons[i_plot_0:], results['mean_trend'][1,i_plot_0:], c='k', marker='d', label='myocvxMPC')
    ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[1,i_plot_0:])*scale, (upperbound[1,i_plot_0:])*scale, alpha=0.2, facecolor='k')
    ax1[n_case].plot(planning_horizons[i_plot_0:], results['mean_trend'][2,i_plot_0:], c='indianred', marker='s', label='artMPC')
    ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[2,i_plot_0:])*scale, (upperbound[2,i_plot_0:])*scale, alpha=0.2, facecolor='indianred')
    ax1[n_case].plot(planning_horizons[i_plot_0:], results['mean_trend'][3,i_plot_0:], c='C1', marker = '^', label='artMPC_dag')
    ax1[n_case].fill_between(planning_horizons[i_plot_0:], (lowerbound[3,i_plot_0:])*scale, (upperbound[3,i_plot_0:])*scale, alpha=0.2, facecolor='C1')
    if n_case == 0:
        ax1[n_case].legend(loc='upper left')#, bbox_to_anchor=(0.5, -0.1), ncol=3)
    if n_case+1 == len(case_study):
        ax1[n_case].set_xlabel('$\mathrm{H}_{\mathrm{plan}}$')
    else:
        ax1[n_case].set_xticklabels([])
    ax1[n_case].set_ylabel('Computational \n times [s]', fontsize=13)
    ax1[n_case].set_title(case_study[n_case])
    ax1[n_case].tick_params(axis='y')
    '''ax1.set_yscale('log')'''
fig.savefig('cost_per_applications.svg')

In [ ]:
aggr_curves = {
    'cvxMPC' : np.vstack((res[case_study[0]]['mean_trend'][0], res[case_study[1]]['mean_trend'][0], res[case_study[2]]['mean_trend'][0])),
    'myocvxMPC' : np.vstack((res[case_study[0]]['mean_trend'][1], res[case_study[1]]['mean_trend'][1], res[case_study[2]]['mean_trend'][1])),
    'artMPC' : np.vstack((res[case_study[0]]['mean_trend'][2], res[case_study[1]]['mean_trend'][2], res[case_study[2]]['mean_trend'][2])),
    'artMPC_dag' : np.vstack((res[case_study[0]]['mean_trend'][3], res[case_study[1]]['mean_trend'][3], res[case_study[2]]['mean_trend'][3]))
}
mean_trend_aggr = {
    'cvxMPC' : aggr_curves['cvxMPC'].mean(axis=0),
    'myocvxMPC' : aggr_curves['myocvxMPC'].mean(axis=0),
    'artMPC' : aggr_curves['artMPC'].mean(axis=0),
    'artMPC_dag' : aggr_curves['artMPC_dag'].mean(axis=0)
}
std_trend_aggr = {
    'cvxMPC' : aggr_curves['cvxMPC'].std(axis=0),
    'myocvxMPC' : aggr_curves['myocvxMPC'].std(axis=0),
    'artMPC' : aggr_curves['artMPC'].std(axis=0),
    'artMPC_dag' : aggr_curves['artMPC_dag'].std(axis=0)
}

lowerbound = np.zeros((4,len(planning_horizons)))
upperbound = np.zeros((4,len(planning_horizons)))
for i in range(lowerbound.shape[0]):
    key = list(mean_trend_aggr.keys())[i]
    lowerbound[i] = mean_trend_aggr[key] - std_trend_aggr[key]
    upperbound[i] = mean_trend_aggr[key] + std_trend_aggr[key]

sns_red = (0.7686274509803922, 0.3058823529411765, 0.3215686274509804)
l_width = 2.5
plt.style.use('seaborn-v0_8-colorblind')
fig, ax1 = plt.subplots(figsize=(8,4))
box = ax1.get_position()
ax1.set_position([box.x0, box.y0 + box.height * 0.15, box.width, box.height * 0.85])
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['cvxMPC'][i_plot_0:], c='C0', linewidth=l_width, marker='o', label='REL-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[0,i_plot_0:])*scale, (upperbound[0,i_plot_0:])*scale, alpha=0.1, facecolor='C0')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['myocvxMPC'][i_plot_0:], c='k', linewidth=l_width, marker='d', label='DIST-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[1,i_plot_0:])*scale, (upperbound[1,i_plot_0:])*scale, alpha=0.1, facecolor='k')
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC'][i_plot_0:], c=sns_red, linewidth=l_width, marker='s', label='TTO-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[2,i_plot_0:])*scale, (upperbound[2,i_plot_0:])*scale, alpha=0.15, facecolor=sns_red)
ax1.plot(planning_horizons[i_plot_0:], mean_trend_aggr['artMPC_dag'][i_plot_0:], c='C1', linewidth=l_width, marker = '^', label='FT-TTO-RHC')
ax1.fill_between(planning_horizons[i_plot_0:], (lowerbound[3,i_plot_0:])*scale, (upperbound[3,i_plot_0:])*scale, alpha=0.15, facecolor='C1')
'''ax1.plot(planning_horizons[i_plot_0:], np.zeros_like(planning_horizons[i_plot_0:])*scale,'k--',label='Lower bound')'''
ax1.legend(loc='upper left', fontsize=12)#, bbox_to_anchor=(0.5, 1.15))
ax1.set_xlabel('Planning horizon', fontsize=13)
ax1.set_ylabel('Computational \n times [s]', fontsize=13)
ax1.set_xticks(np.arange(10,101,10))
ax1.tick_params(axis='y')
#ax1.set_yscale('log')
fig.savefig('closed_loop_time_cross_applications.svg')

In [ ]:
sns.color_palette()[2]

In [ ]:
mean_time = [[[td_h.mean() for td_h in tdm_method] for tdm_method in results['time_dataset']] for results in np.array(list(res.values()))[[0,2]]]
std_time = [[[td_h.std() for td_h in tdm_method] for tdm_method in results['time_dataset']] for results in np.array(list(res.values()))[[0,2]]]
for n_case in range(len(an)):
    for i in range(4):
        if len(mean_time[n_case][i]) == 0:
            mean_time[n_case][i] = [np.nan]*len(H_labels)
            std_time[n_case][i] = [np.nan]*len(H_labels)
mean_time = np.array(mean_time)
std_time = np.array(std_time)

for i in range(4):
    

fig, ax = plt.subplots()
ax.plot(np.arange(10,101,10),mean_time[0][0],c='C0',label='cvxMPC')
ax.fill_between(np.arange(10,101,10),lowerbound[0],upperbound[0],facecolor='C0',alpha=0.1)
'''ax.plot(np.arange(10,101,10),mean_time[1],label='myocvxMPC')
ax.plot(np.arange(10,101,10),mean_time[2],label='artMPC')
ax.fill_between(np.arange(10,101,10),lowerbound[2],upperbound[2],facecolor='C2',alpha=0.2)'''
ax.plot(np.arange(10,101,10),mean_time[0][3],c='C1',label='artMPC_dag')
ax.fill_between(np.arange(10,101,10),lowerbound[3],upperbound[3],facecolor='C1',alpha=0.15)
#ax.plot(np.arange(10,101,10), 0.1*np.ones(10,),'k--',label='0.1 s')
ax.set_xlabel('$H_{plan}$')
ax.set_ylabel('Computational time [s]')
ax.legend(loc='upper left')
ax.set_title(case_study[n_case])
#ax.set_yscale('log')
fig.savefig('')
plt.show()

In [ ]:
mean_time.shape